In [1]:
from dwave.cloud import Client

client = Client.from_config(token='WASE-c2ee8615ffb9fc1953570636217c253aa213dc34')
print(client.get_solvers())

[StructuredSolver(id='Advantage_system1.1'), StructuredSolver(id='DW_2000Q_6'), StructuredSolver(id='DW_2000Q_VFYC_6')]


In [2]:
from dwave.cloud import Client

with Client.from_config() as client:
    solver = client.get_solver()
    print('solver: ' + solver.data['id'])

    u, v = next(iter(solver.edges))
    print('edges: ' + str([u, v]))
    Q = { 
        (u, u): -3, 
        (u, v): 4,
        (v, u): 0,
        (v, v): -4, 
    }   

    computation = solver.sample_qubo(Q, num_reads=5)

    for sample in computation.samples:
        print('result: ' + str([int(sample[u]), int(sample[v])]))

    print('timing: ' + str(computation.timing))

solver: DW_2000Q_6
edges: [711, 707]
result: [0, 1]
timing: {'qpu_sampling_time': 1195, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 198, 'qpu_access_time': 11961, 'qpu_access_overhead_time': 1779, 'qpu_programming_time': 10767, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 110, 'post_processing_overhead_time': 110, 'total_real_time': 11961, 'run_time_chip': 1195, 'anneal_time_per_run': 20, 'readout_time_per_run': 198}


In [20]:
solver.edges

{(711, 707),
 (1634, 1638),
 (416, 288),
 (587, 590),
 (1099, 1102),
 (877, 872),
 (1389, 1384),
 (642, 645),
 (1820, 1818),
 (260, 256),
 (1388, 1396),
 (601, 606),
 (1113, 1118),
 (963, 835),
 (855, 849),
 (497, 501),
 (1551, 1547),
 (1950, 1945),
 (1538, 1666),
 (1724, 1721),
 (1426, 1428),
 (1118, 1112),
 (1107, 1109),
 (1197, 1195),
 (1397, 1395),
 (1790, 1786),
 (506, 508),
 (1505, 1510),
 (99, 103),
 (388, 386),
 (1660, 1658),
 (619, 620),
 (1131, 1132),
 (1949, 1945),
 (1083, 1085),
 (653, 650),
 (1165, 1162),
 (697, 702),
 (1913, 1918),
 (1159, 1152),
 (1613, 1605),
 (1391, 1387),
 (1544, 1672),
 (2037, 2033),
 (500, 496),
 (840, 845),
 (350, 342),
 (1196, 1193),
 (1799, 1794),
 (594, 599),
 (1054, 1051),
 (1528, 1533),
 (490, 492),
 (383, 379),
 (1763, 1764),
 (1409, 1537),
 (1579, 1581),
 (1869, 1867),
 (590, 586),
 (681, 686),
 (1102, 1098),
 (49, 52),
 (349, 345),
 (1509, 1505),
 (120, 127),
 (484, 480),
 (1135, 1143),
 (1443, 1571),
 (1030, 1025),
 (777, 649),
 (1289, 116

In [3]:
# 問題設定
NUM_VER = 6
vertices = list(range(NUM_VER))
edges = [(0,1), (0,4), (0,5), (1,2), (1,3), (3,4), (4,5)]

In [15]:
v

707

In [4]:
from pyqubo import Array, Constraint, Placeholder, solve_qubo

# BINARY変数
x = Array.create('x', shape=NUM_VER, vartype='BINARY')

# QUBO形式で定式化
H_cover = Constraint(sum((1-x[u])*(1-x[v]) for (u,v) in edges), "cover")
H_vertices = sum(x)
H = H_vertices + Placeholder("cover") * H_cover

# モデルをコンパイル
model = H.compile()

# プレースホルダーに具体的な数値を当てはめて、QUBOを作成
feed_dict = {"cover": 1.0}
qubo, offset = model.to_qubo(feed_dict=feed_dict)

In [5]:
# SAで解を探索する
raw_solution = solve_qubo(qubo)

# 得られた結果をデコードする
decoded_solution, broken, energy = model.decode_solution(raw_solution, vartype="BINARY", feed_dict=feed_dict)


In [6]:
print(decoded_solution)

{'x': {0: 0, 1: 1, 2: 0, 3: 0, 4: 1, 5: 0}}


In [7]:
endpoint='https://cloud.dwavesys.com/sapi/'
token='WASE-c2ee8615ffb9fc1953570636217c253aa213dc34'
solver= client.get_solver()
solver_name=solver.data['id']

In [8]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler(endpoint=endpoint, token=token, solver=solver_name))
response = sampler.sample_qubo(qubo)

In [9]:
response

SampleSet(rec.array([([0, 1, 0, 0, 1, 1], -4., 1, 0.)],
          dtype=[('sample', 'i1', (6,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), ['x[0]', 'x[1]', 'x[2]', 'x[3]', 'x[4]', 'x[5]'], {'timing': {'qpu_sampling_time': 239, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 198, 'qpu_access_time': 11025, 'qpu_access_overhead_time': 1133, 'qpu_programming_time': 10786, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 507, 'post_processing_overhead_time': 507, 'total_real_time': 11025, 'run_time_chip': 239, 'anneal_time_per_run': 20, 'readout_time_per_run': 198}, 'problem_id': '24ab1215-8632-4f26-b0c6-5947445fc541'}, 'BINARY')